<h3>Scrape Case Details</h3>
From each case's page on CaseNet, collect the following information:
<ul>
    - Judge name <br>
    - Disposed status <br>
    - Charge Description <br>
    - Ordinance Number <br>
    - Law Enforcement Agency <br>
    - Civil judgments <br>
    - Garnishments and executions <br>
    </ul>

In [23]:
#import the necessary python packages
import sys

selenium_path = r'C:\Users\Kimbe\anaconda3\Lib\site-packages\selenium'
PYSOLR_PATH = r'c:\users\kimbe\appdata\local\programs\python\python312\lib\site-packages'

if not PYSOLR_PATH in sys.path:
    sys.path.append(PYSOLR_PATH)

if not selenium_path in sys.path:
    sys.path.append(selenium_path)

import pandas
import numpy
import requests
import bs4
import re
import undetected_chromedriver as uc
import selenium
import os
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime
import re

In [24]:
#Define a function to parse the case header HTML text
def parse_case_header(text, case_number):
    soup = bs4.BeautifulSoup(text, 'html.parser')
    
    try:
        objects = str(soup.find_all(attrs={"class": 'col-lg-5 col-md-6 col-sm-6 col-xs-12 odd'})).replace('<strong>', '').replace('<div class="col-lg-5 col-md-6 col-sm-6 col-xs-12 odd', '').replace('">', '').replace('</strong>', '').replace('</div>', '').replace('\n', '').replace("[", '').replace(']', '')
        values = str(soup.find_all(attrs={"class": 'col-lg-7 col-md-6 col-sm-6 col-xs-12 even'})).replace('<div class="col-lg-7 col-md-6 col-sm-6 col-xs-12 even', '').replace('">', '').replace('</div>', '').replace('\n', '').replace("[", '').replace(']', '')
 
        column_names = [x.strip() for x in objects.split(',')]

        case_data = [x.strip() for x in values.split(',')]

        #Clean up surname issues
        if 'Judge/Commissioner Assigned' in column_names:
            case_data[0] = case_data[0] + ', ' + case_data[1]
            case_data.pop(1)

        if 'Judge/Commissioner At Disposition' in column_names:
            first_name_index = len(case_data) - 1
            last_name_index = first_name_index - 1
            case_data[last_name_index] = case_data[last_name_index] + ', ' + case_data[first_name_index]
            case_data.pop(first_name_index)

        if 'Financial Information' in column_names:
            fin_info_index = len(case_data)
            column_names.pop(fin_info_index)

        case_details = pandas.DataFrame([case_data], columns = [column_names])
        
        #When a case doesn't have a field for Judge/Commissioner At Disposition, add a 
        #column containing NaN
        if 'Judge/Commissioner At Disposition' in case_details.columns:
            numpy.nan
        else:
            case_details['Judge/Commissioner At Disposition'] = numpy.nan
            
        #Create a column for case number
        case_details['case_number'] = case_number
        return case_details
    except:
        print("ERROR on case_number " + case_number)
        return

In [29]:
#read the links and identify the highest value in the index
#cases = pandas.read_csv(r"C:\Users\Kimbe\Documents\Data Science\Sample Data - Week of Jan 2, 2025.csv")
cases = pandas.read_csv(r"data\case_details_data\case_details_2024-12-22.csv")
stop = len(cases) - 1
dates = cases['date_filed'].drop_duplicates().reset_index()
error_cases = []

In [30]:
#Launch the headless browser
driver = uc.Chrome()

for i in range(0, 6):
    #Get all cases for a particular day
    cases_subset = cases.loc[cases['date_filed'] == dates.iloc[i,1]]
    date_name = str(datetime.datetime.strptime(dates.iloc[i,1], '%m/%d/%Y').strftime('%Y-%m-%d'))
    #Get the highest index value
    stop = max(cases_subset.index)
    #Loop through the list of cases and scrape the header data. Save the data in a pandas dataframe.
    #for j in range(0, stop):
    for j in range(0, 20):
            try:
                driver.get(cases_subset.iloc[j, 5])
                case_number = cases_subset.iloc[j, 1]

                #Give the page time to load
                time.sleep(2)
                html_data = driver.page_source
                output = parse_case_header(html_data, case_number)

                if "case_headers" in locals():
                    case_headers = case_headers.append(output)
                else:
                    case_headers = output
            except:
                print('error on case number ' + str(case_number))
                error_cases.append(str(case_number))
    file_name = 'data\\header_data\\' + date_name + '_case_header_details.csv'
    case_headers.to_csv(file_name, index = False)
    del(case_headers)


C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\A

C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\A

C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\A

C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\AppData\Local\Temp/ipykernel_14432/4137862153.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  case_headers = case_headers.append(output)
C:\Users\Kimbe\A

In [6]:
print(output)

  Judge/Commissioner Assigned  Date Filed           Location  \
0      STRUM, STEPHEN MICHAEL  01/07/2025  Clayton Municipal   

                            Case Type   Disposition  \
0  Municipal/ Ordinance - see Charges  Not Disposed   

  Judge/Commissioner At Disposition case_number  
0                               NaN   191243981  


In [16]:
error_cases

[]